Hadoop/MapReduce -- WordCount en Python (Hadoop Streaming)
===

* *30 min* | Última modificación: Junio 22, 2019

## Definición del problema

Se desea contar la frecuencia de ocurrencia de palabras en un conjunto de documentos. Debido a los requerimientos de diseño (gran volúmen de datos y tiempos rápidos de respuesta) se desea implementar una arquitectura Big Data. Se desea implementar la solución en **Python** y ejecutar Hadoop en **modo standalone**.

A continuación se generarán tres archivos de prueba para probar el sistema.

In [1]:
## Se crea el directorio de entrada
!rm -rf input output
!mkdir input

In [2]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [3]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [4]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Writing input/text2.txt


## Solución

### Prueba de la implementación fuera de Hadoop

Para realizar la implementación solicitada se usará Hadoop Streaming, el cual permite la implementación de aplicaciones MapReduce en cualquier lenguaje que soporte la creación de scripts. Para este caso se ejemplificará la implementación en Python. Hadoop streaming debe su nombre a que usa los streamings de Unix para la entrada y salida en texto (stdin, stdout y stderr respectivamente). El sistema lee y escribe líneas de texto una a una a los respectivos flujos de entrada y salida, tal como ocurre de forma estándar en las herramientas de la línea de comandos de Linux. 

La implementación requiere un programa para realizar el mapeo y otro para la reducción que se ejecutan independientemente; Hadoop se encarga de la coordinación entre ellos. El intercambio de información se hace a traves de texto, que es el lenguaje universal para intercambio de información. 

#### Paso 1

Se implementa la función map en Python y se guarda en el archivo `mapper.py`.

In [5]:
%%writefile mapper.py
#! /usr/bin/env python

##
## Esta es la función que mapea la entrada a parejas (clave, valor)
##
import sys
if __name__ == "__main__": 
    
    ##
    ## itera sobre cada linea de código recibida
    ## a través del flujo de entrada
    ##
    for line in sys.stdin:
        
        ##
        ## genera las tuplas palabra \tabulador 1
        ## ya que es un conteo de palabras
        ##
        for word in line.split(): 
                   
            ##
            ## escribe al flujo estándar de salida
            ##
            sys.stdout.write("{}\t1\n".format(word))
            

Writing mapper.py


#### Paso 2

In [6]:
## El programa anterior se hace ejecutable
!chmod +x mapper.py 

In [7]:
## la salida de la función anterior es:
!cat ./input/text*.txt | ./mapper.py | head

Analytics	1
is	1
the	1
discovery,	1
interpretation,	1
and	1
communication	1
of	1
meaningful	1
patterns	1


### Paso 3

Se implementa la función `reduce` y se salva en el archivo `reducer.py`.

In [8]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

##
## Esta función reduce los elementos que 
## tienen la misma clave
##

if __name__ == '__main__': 
  
    curkey = None
    total = 0
    
    ##
    ## cada linea de texto recibida es una 
    ## entrada clave \tabulador valor
    ##
    for line in sys.stdin:
        
        key, val = line.split("\t") 
        val = int(val)
        
        if key == curkey: 
            ##
            ## No se ha cambiado de clave. Aca se 
            ## acumulan los valores para la misma clave.
            ##
            total += val  
        else:
            ##
            ## Se cambio de clave. Se reinicia el
            ## acumulador.
            ##
            if curkey is not None:
                ##
                ## una vez se han reducido todos los elementos
                ## con la misma clave se imprime el resultado en
                ## el flujo de salida
                ##
                sys.stdout.write("{}\t{}\n".format(curkey, total)) 
            
            curkey = key
            total = val

Writing reducer.py


#### Paso 4

In [9]:
## El archivo se hace ejecutable
!chmod +x reducer.py

#### Paso 5

Se realiza la prueba de la implementación en el directorio actual antes de realizar la ejecución en el modo pseudo-distribuido

In [10]:
##
## La función sort hace que todos los elementos con 
## la misma clave queden en lineas consecutivas.
## Hace el papel del módulo Shuffle & Sort
##
!cat ./input/text*.txt | ./mapper.py | sort | ./reducer.py | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	2
Especially	1
Organizations	1
Since	1
Specifically,	1


### Ejecución en Hadoop

Una vez se tienen las versiones anteriores funcionando, se puede proceder a ejecutar la tarea directamente en Hadoop. 

In [11]:
## Se elimina el directorio de salida si existe
!rm -rf output

#### Paso 1 (solo en modo pseudo-distribuido

Se copian los archivos de entrada al HDFS. Como se realizará la ejecución en modo standalone no es necesario, ya que Hadoop corre en el sistema local de archivos.


Crea el directorio para los archivos de entrada


    !$HADOOP_HOME/bin/hadoop fs -mkdir input

Copia el contenido de la carpeta input a la carpeta input en el HDFS

    !$HADOOP_HOME/bin/hadoop fs -put  input/* input

Verifica el contenido del directorio

    !$HADOOP_HOME/bin/hadoop fs -ls input/*

#### Paso 2

Se ejecuta Hadoop Streaming en modo pseudo-distribuido. 

In [12]:
##
## Se ejecuta en Hadoop.
##   -input: archivo de entrada
##   -output: directorio de salida
##   -maper: programa que ejecuta el map
##   -reducer: programa que ejecuta la reducción
##
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar -input input -output output  -mapper mapper.py -reducer reducer.py

In [13]:
## contenido del directorio con los 
## resultados de la corrida
!$HADOOP_HOME/bin/hadoop fs -ls output

Found 2 items
-rw-r--r--   1 jdvelasq admin          0 2018-10-15 20:49 output/_SUCCESS
-rw-r--r--   1 jdvelasq admin       1640 2018-10-15 20:49 output/part-00000


In [14]:
## se visualiza el archivo con los
## resultados de la corrida
!$HADOOP_HOME/bin/hadoop fs -cat output/part-00000 | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


### Limpieza

In [15]:
!rm reducer.py mapper.py
!rm -rf input output

En modo pseudo-distribuido se ejecutarían adicionalmente

    $HADOOP_HOME/bin/hadoop fs -rm input/* output/*
    
    $HADOOP_HOME/bin/hadoop fs -rmdir input output

## Notas.

**Combiners.--** Los combiners son *reducers* que se ejecutan sobre los resultdos que produce cada mapper antes de pasar al modulo de suffle-&-sort, con el fin de reducir la carga computacional. Suelen ser identicos a los *reducers*. Una llamada típica sería:


     $HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
        -input input \
        -output output  \
        -mapper mapper.py \
        -reducer reducer.py \
        -combiner combiner.py


**Partitioners.--** Son rutinas que controlan como se enviar las parejas (clave, valor) a cada reducers, tal que elementos con la misma clave son enviados al mismo reducer. 

**Job Chain.--** Se refiere al encadenamiento de varias tareas cuando el cómputo que se desea realizar es muy complejo para que pueda realizarse en un MapReduce.